# Initial Preprocessing of the data

In [297]:
# Imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import json
import ast
from datetime import datetime
import re
from sklearn.preprocessing import MultiLabelBinarizer

In [260]:
# Creating dataframes for the respective datasets
credits = pd.read_csv('../data/credits.csv')
keywords = pd.read_csv('../data/keywords.csv')
movies = pd.read_csv('../data/movies_metadata.csv', low_memory=False)
ratings = pd.read_csv('../data/ratings.csv')

In [261]:
print("Shape of credits: ")
print(credits.shape)
print("Shape of keywords:")
print(keywords.shape)
print("Shape of movies:")
print(movies.shape)
print("Shape of ratings:")
print(ratings.shape)

Shape of credits: 
(45476, 3)
Shape of keywords:
(46419, 2)
Shape of movies:
(45466, 24)
Shape of ratings:
(26024289, 4)


In [262]:
movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45466 entries, 0 to 45465
Data columns (total 24 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   adult                  45466 non-null  object 
 1   belongs_to_collection  4494 non-null   object 
 2   budget                 45466 non-null  object 
 3   genres                 45466 non-null  object 
 4   homepage               7782 non-null   object 
 5   id                     45466 non-null  object 
 6   imdb_id                45449 non-null  object 
 7   original_language      45455 non-null  object 
 8   original_title         45466 non-null  object 
 9   overview               44512 non-null  object 
 10  popularity             45461 non-null  object 
 11  poster_path            45080 non-null  object 
 12  production_companies   45463 non-null  object 
 13  production_countries   45463 non-null  object 
 14  release_date           45379 non-null  object 
 15  re

In [263]:
credits.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45476 entries, 0 to 45475
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   cast    45476 non-null  object
 1   crew    45476 non-null  object
 2   id      45476 non-null  int64 
dtypes: int64(1), object(2)
memory usage: 1.0+ MB


In [264]:
keywords.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 46419 entries, 0 to 46418
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        46419 non-null  int64 
 1   keywords  46419 non-null  object
dtypes: int64(1), object(1)
memory usage: 725.4+ KB


### Dataframe cleaning and wrangling

In [265]:
# First dropping columns that seem redundant/irrelevant for further analysis - keeping id as a reference between dataframes
columns_to_drop = ['adult', 'belongs_to_collection', 'imdb_id', 'original_title', 'homepage', 'poster_path',
    'status', 'video', 'tagline', 'overview', 'production_companies', 'production_countries', 'spoken_languages']

movies = movies.drop(columns=columns_to_drop)

In [266]:
movies.shape

(45466, 11)

In [267]:
# Finding null-values
movies.isnull().sum()

budget                 0
genres                 0
id                     0
original_language     11
popularity             5
release_date          87
revenue                6
runtime              263
title                  6
vote_average           6
vote_count             6
dtype: int64

In [268]:
# Removing null from the dataframe
movies.dropna(inplace=True)

In [269]:
movies.shape

(45119, 11)

In [270]:
movies.drop_duplicates()

,budget,genres,id,original_language,popularity,release_date,revenue,runtime,title,vote_average,vote_count
0,30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",862,en,21.946943,1995-10-30,373554033.0,81.0,Toy Story,7.7,5415.0
1,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",8844,en,17.015539,1995-12-15,262797249.0,104.0,Jumanji,6.9,2413.0
2,0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",15602,en,11.7129,1995-12-22,0.0,101.0,Grumpier Old Men,6.5,92.0
3,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",31357,en,3.859495,1995-12-22,81452156.0,127.0,Waiting to Exhale,6.1,34.0
4,0,"[{'id': 35, 'name': 'Comedy'}]",11862,en,8.387519,1995-02-10,76578911.0,106.0,Father of the Bride Part II,5.7,173.0
...,...,...,...,...,...,...,...,...,...,...,...
45460,0,"[{'id': 18, 'name': 'Drama'}, {'id': 28, 'name...",30840,en,5.683753,1991-05-13,0.0,104.0,Robin Hood,5.7,26.0
45462,0,"[{'id': 18, 'name': 'Drama'}]",111109,tl,0.178241,2011-11-17,0.0,360.0,Century of Birthing,9.0,3.0
45463,0,"[{'id': 28, 'name': 'Action'}, {'id': 18, 'nam...",67758,en,0.903007,2003-08-01,0.0,90.0,Betrayal,3.8,6.0
45464,0,[],227506,en,0.003503,1917-10-21,0.0,87.0,Satan Triumphant,0.0,0.0


In [271]:
movies.info()

<class 'pandas.core.frame.DataFrame'>
Index: 45119 entries, 0 to 45465
Data columns (total 11 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   budget             45119 non-null  object 
 1   genres             45119 non-null  object 
 2   id                 45119 non-null  object 
 3   original_language  45119 non-null  object 
 4   popularity         45119 non-null  object 
 5   release_date       45119 non-null  object 
 6   revenue            45119 non-null  float64
 7   runtime            45119 non-null  float64
 8   title              45119 non-null  object 
 9   vote_average       45119 non-null  float64
 10  vote_count         45119 non-null  float64
dtypes: float64(4), object(7)
memory usage: 4.1+ MB


In [272]:
# As id and bduget is not numeric, I'll convert these
# Converting the release_date to datetime as well
movies['id'] = pd.to_numeric(movies['id'], errors='coerce')
movies['budget'] = pd.to_numeric(movies['budget'], errors='coerce')
movies['release_date'] = pd.to_datetime(movies['release_date'], errors='coerce')

In [273]:
movies.info()

<class 'pandas.core.frame.DataFrame'>
Index: 45119 entries, 0 to 45465
Data columns (total 11 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   budget             45119 non-null  int64         
 1   genres             45119 non-null  object        
 2   id                 45119 non-null  int64         
 3   original_language  45119 non-null  object        
 4   popularity         45119 non-null  object        
 5   release_date       45119 non-null  datetime64[ns]
 6   revenue            45119 non-null  float64       
 7   runtime            45119 non-null  float64       
 8   title              45119 non-null  object        
 9   vote_average       45119 non-null  float64       
 10  vote_count         45119 non-null  float64       
dtypes: datetime64[ns](1), float64(4), int64(2), object(4)
memory usage: 4.1+ MB


In [274]:
movies.head(3)

,budget,genres,id,original_language,popularity,release_date,revenue,runtime,title,vote_average,vote_count
0,30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",862,en,21.946943,1995-10-30,373554033.0,81.0,Toy Story,7.7,5415.0
1,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",8844,en,17.015539,1995-12-15,262797249.0,104.0,Jumanji,6.9,2413.0
2,0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",15602,en,11.7129,1995-12-22,0.0,101.0,Grumpier Old Men,6.5,92.0


It's important that all movies have received votes, and to make sure they're somewhat qualitative, I'll remove all rows where vote_count is < 10

In [275]:
movies = movies[movies['vote_count'] >= 10].reset_index(drop=True)



In [276]:
movies.shape

(22914, 11)

In [277]:
# Convert 'id' columns to numeric (if not already)
keywords['id'] = pd.to_numeric(keywords['id'], errors='coerce')
credits['id'] = pd.to_numeric(credits['id'], errors='coerce')

# Drop rows with NaN ids (if any)
keywords = keywords.dropna(subset=['id'])
credits = credits.dropna(subset=['id'])

# Reset index after dropping
keywords = keywords.reset_index(drop=True)
credits = credits.reset_index(drop=True)

In [278]:
# The keywords and credits dataframes have column with stringified lists of dictionaires.
# I'll convert these to Python objects for future analysis
# Parse the 'keywords' column
keywords['keywords'] = keywords['keywords'].apply(ast.literal_eval)

# Parse the 'cast' and 'crew' columns in credits
credits['cast'] = credits['cast'].apply(ast.literal_eval)
credits['crew'] = credits['crew'].apply(ast.literal_eval)

In [279]:
keywords.shape

(46419, 2)

In [280]:
# Checking for duplicate IDs in keywords 
duplicate_ids = keywords['id'].duplicated().sum()
print(f"Number of duplicate IDs in keywords: {duplicate_ids}")

Number of duplicate IDs in keywords: 987


In [281]:
keywords = keywords.drop_duplicates('id')

In [282]:
keywords.shape

(45432, 2)

In [283]:
# Keeping only top 5 actors from the cast, and only the directors from the crew
def extract_top_cast(cast_data, top=5):
    try:
        # Sort by 'order' and take top N
        sorted_cast = sorted(cast_data, key=lambda x: x.get('order', 999))[:top]
        return [actor['name'] for actor in sorted_cast]
    except (ValueError, TypeError):
        return []

credits['top_cast'] = credits['cast'].apply(extract_top_cast)

In [284]:
credits.head(3)

,cast,crew,id,top_cast
0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...",862,"[Tom Hanks, Tim Allen, Don Rickles, Jim Varney..."
1,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...",8844,"[Robin Williams, Jonathan Hyde, Kirsten Dunst,..."
2,"[{'cast_id': 2, 'character': 'Max Goldman', 'c...","[{'credit_id': '52fe466a9251416c75077a89', 'de...",15602,"[Walter Matthau, Jack Lemmon, Ann-Margret, Sop..."


In [285]:
def extract_director(crew_data):
    try:
        directors = [member['name'] for member in crew_data if member.get('job') == 'Director']
        return directors[0] if directors else None
    except (ValueError, TypeError):
        return None
    
credits['director'] = credits['crew'].apply(extract_director)

In [286]:
credits.head(3)

,cast,crew,id,top_cast,director
0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...",862,"[Tom Hanks, Tim Allen, Don Rickles, Jim Varney...",John Lasseter
1,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...",8844,"[Robin Williams, Jonathan Hyde, Kirsten Dunst,...",Joe Johnston
2,"[{'cast_id': 2, 'character': 'Max Goldman', 'c...","[{'credit_id': '52fe466a9251416c75077a89', 'de...",15602,"[Walter Matthau, Jack Lemmon, Ann-Margret, Sop...",Howard Deutch


In [287]:
credits = credits.drop(columns=['cast', 'crew'])

In [288]:
credits.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45476 entries, 0 to 45475
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        45476 non-null  int64 
 1   top_cast  45476 non-null  object
 2   director  44589 non-null  object
dtypes: int64(1), object(2)
memory usage: 1.0+ MB


In [289]:
credits.shape

(45476, 3)

In [290]:
# Checking for duplicate IDs in credits 
duplicate_ids = credits['id'].duplicated().sum()
print(f"Number of duplicate IDs in credits: {duplicate_ids}")

Number of duplicate IDs in credits: 44


In [291]:
credits = credits.drop_duplicates('id')

In [292]:
credits.shape

(45432, 3)

In [293]:
# Merging the dataframes using the id column. Using a left join to only apply it to the movies rows
movies_keywords = pd.merge(movies, keywords, left_on='id', right_on='id', how='left')

movies_full = pd.merge(movies_keywords, credits, left_on='id', right_on='id', how='left')

In [294]:
movies_full.shape

(22914, 14)

In [295]:
movies_full.head(3)

,budget,genres,id,original_language,popularity,release_date,revenue,runtime,title,vote_average,vote_count,keywords,top_cast,director
0,30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",862,en,21.946943,1995-10-30,373554033.0,81.0,Toy Story,7.7,5415.0,"[{'id': 931, 'name': 'jealousy'}, {'id': 4290,...","[Tom Hanks, Tim Allen, Don Rickles, Jim Varney...",John Lasseter
1,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",8844,en,17.015539,1995-12-15,262797249.0,104.0,Jumanji,6.9,2413.0,"[{'id': 10090, 'name': 'board game'}, {'id': 1...","[Robin Williams, Jonathan Hyde, Kirsten Dunst,...",Joe Johnston
2,0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",15602,en,11.7129,1995-12-22,0.0,101.0,Grumpier Old Men,6.5,92.0,"[{'id': 1495, 'name': 'fishing'}, {'id': 12392...","[Walter Matthau, Jack Lemmon, Ann-Margret, Sop...",Howard Deutch


In [296]:
# Extract genre names from the genres column
def extract_genres(genre_data):
    if isinstance(genre_data, str):
        try:
            genres = ast.literal_eval(genre_data)
            return [genre['name'] for genre in genres]
        except (ValueError, SyntaxError):
            return []
    elif isinstance(genre_data, list):
        return [genre['name'] for genre in genre_data]
    return []

movies_full['genre_names'] = movies_full['genres'].apply(extract_genres)